### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
    !pip install synthetic-data-kit==0.0.3
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
    !pip install synthetic-data-kit==0.0.3


In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

## Start VLM server

In [ ]:
from unsloth.dataprep import SyntheticDataKit

generator = SyntheticDataKit.from_pretrained(
    # Choose any model from https://huggingface.co/unsloth
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048, # Longer sequence lengths will be slower!
)

Unsloth: Your GPU cannot handle sequence lengths of 256 due to limited GPU memory.
Unsloth: Your GPU can only handle approximately the maximum sequence length of 256.
Unsloth: Using dtype = torch.float16 for vLLM.
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 11.91%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 256. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 0.0 GB. Also swap space = 0 GB.
vLLM STDOUT: INFO 05-21 18:36:42 [__init__.py:239] Automatically detected platform cuda.
vLLM STDOUT: INFO 05-21 18:36:53 [api_server.py:1043] vLLM API server version 0.8.5.post1
vLLM STDOUT: INFO 05-21 18:36:53 [api_server.py:1044] args: Namespace(subparser='serve', model_tag='unsloth/Llama-3.2-3B-Instruct', config='', host=None, port=8000, uvicorn_log_level='info', disable_uvicorn_access_log=False, allow_credentials=False, allowed_origins=['*'], allowed_meth

## Generate QA Pairs + Auto clean data

In [ ]:
generator.prepare_qa_generation(
    output_folder = "data", # Output location of synthetic data
    temperature = 0.7, # Higher temp makes more diverse datases
    top_p = 0.95,
    overlap = 64, # Overlap portion during chunking
    max_generation_tokens = 512, # Can increase for longer QA pairs
)

### Sanity Checks

In [ ]:
!synthetic-data-kit system-check

L VLLM server is not available at http://localhost:8000/v1
Error: HTTPConnectionPool(host='localhost', port=8000): Read timed out. (read 
timeout=2)

To start the server, run:
vllm serve meta-llama/Llama-3.3-70B-Instruct --port 8000
⠼ Checking VLLM server at http://localhost:8000/v1...


In [ ]:

import requests

url = "http://localhost:8000/v1/chat/completions"

payload = {
    "model": "unsloth/Llama-3.2-3B-Instruct",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
    "temperature": 0.7,
    "max_tokens": 100
}

response = requests.post(url, json=payload)

# Pretty-print the result
if response.status_code == 200:
    print(response.json()["choices"][0]["message"]["content"])
else:
    print(f"Error {response.status_code}: {response.text}")


### Ingest

In [ ]:
!synthetic-data-kit ingest /content/acuff.pdf
!synthetic-data-kit ingest /content/le.pdf
!synthetic-data-kit ingest /content/paige.pdf
!synthetic-data-kit ingest /content/osborne.pdf

!synthetic-data-kit ingest /content/YT_acuff.pdf
!synthetic-data-kit ingest /content/YT_le.pdf
!synthetic-data-kit ingest /content/YT_paige.pdf
!synthetic-data-kit ingest /content/YT_osborne.pdf

⠹ Processing /content/acuff.pdf...
 Text successfully extracted to data/output/acuff.txt
⠙ Processing /content/le.pdf...
 Text successfully extracted to data/output/le.txt
⠙ Processing /content/paige.pdf...
 Text successfully extracted to data/output/paige.txt
⠹ Processing /content/osborne.pdf...
 Text successfully extracted to data/output/osborne.txt


### Generate

#### Custom Bios text

In [ ]:
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /content/data/output/acuff.txt  \
        --num-pairs 25 \
        --type "qa"
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /content/data/output/le.txt  \
        --num-pairs 25 \
        --type "qa"

Processing 2 chunks to generate QA pairs...
Batch processing complete.
Generated 25 QA pairs total
Saving result to data/generated/acuff_qa_pairs.json
Successfully wrote test file to data/generated/test_write.json
Successfully wrote result to data/generated/acuff_qa_pairs.json
⠧ Generating qa content from /content/data/output/acuff.txt...
 Content saved to data/generated/acuff_qa_pairs.json
Processing 2 chunks to generate QA pairs...
Batch processing complete.
Generated 25 QA pairs total
Saving result to data/generated/le_qa_pairs.json
Successfully wrote test file to data/generated/test_write.json
Successfully wrote result to data/generated/le_qa_pairs.json
⠇ Generating qa content from /content/data/output/le.txt...
 Content saved to data/generated/le_qa_pairs.json
⠋ Generating qa content from /content/data/output/paige.txt...
L Error: Failed to get completion after 3 attempts: 
HTTPConnectionPool(host='localhost', port=8000): Read timed out. (read 
timeout=180)
L Error: VLLM server no

In [ ]:
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /content/data/output/paige.txt  \
        --num-pairs 25 \
        --type "qa"
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /content/data/output/osborne.txt  \
        --num-pairs 25 \
        --type "qa"

### Youtube Transcripts

In [ ]:
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /content/data/output/YT_acuff.txt  \
        --num-pairs 25 \
        --type "qa"
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /content/data/output/YT_le.txt  \
        --num-pairs 25 \
        --type "qa"

In [ ]:
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /content/data/output/YT_paige.txt  \
        --num-pairs 25 \
        --type "qa"
!synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create /content/data/output/YT_osborne.txt  \
        --num-pairs 25 \
        --type "qa"

### Convert to HF format

In [ ]:
qa_pairs_filenames = os.listdir('/content/data/generated')
for filename in qa_pairs_filenames:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        save-as {filename} -f ft

⠋ Converting data/generated/arxiv_org_0_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_0_qa_pairs_ft.json
⠋ Converting data/generated/arxiv_org_1_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_1_qa_pairs_ft.json
⠋ Converting data/generated/arxiv_org_2_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_2_qa_pairs_ft.json


In [ ]:
#generator.cleanup()

Attempting to terminate the VLLM server gracefully...
Server terminated gracefully.
